# Q 2 B. NEURAL NETWORK with 
### 30 Hidden Sigmoid Nodes , 
### 2 HIdden Layers , 
### with and without L2 regularisations and 
### 10 Softmax output nodes 
## -------------COMMON CODE STARTS ----------------##
### - Import Libraries including TensorFlow

In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.reset_default_graph()

## -Read the MNIST Train and Test Data
### - Pick 100 Samples of each digit for Train and Test 
### - Convert the Labels into One Hot Encoded for Softmax Output calculations

In [26]:
data = pd.read_csv('/Users/vikrant/GoogleDrive/CSE555-Intro2Pattern-WenDong/HW-45/mnist_train_nn1.csv')
datatest=pd.read_csv('/Users/vikrant/GoogleDrive/CSE555-Intro2Pattern-WenDong/HW-45/mnist_test_nn1.csv')

tmp=pd.DataFrame()
        
print("GETTING 1000 samples")
for i in range(10):
    loop = data.iloc[np.where(data.iloc[:,0] == i)]
            
            #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
            #https://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe
    loop1 = loop.sample(n=100).reset_index(drop=True)
            
            #print("No. of ROws, columns:" , len(loop1), "for digit:",i)
    tmp = tmp.append(loop1, ignore_index = True)

    
tmptest=pd.DataFrame()
        
print("GETTING 1000 samples")
for i in range(10):
    loop = datatest.iloc[np.where(datatest.iloc[:,0] == i)]
            
            #https://stackoverflow.com/questions/29576430/shuffle-dataframe-rows
            #https://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe
    loop1 = loop.sample(n=100).reset_index(drop=True)
            
            #print("No. of ROws, columns:" , len(loop1), "for digit:",i)
    tmptest = tmptest.append(loop1, ignore_index = True)

    
tmp1=tmp.values
tmptest1=tmptest.values

np.random.shuffle(tmp1)
np.random.shuffle(tmptest1)


images = tmp1[:,1:]
#images = np.multiply(images, 1.0 / 255.0)
imagestest = tmptest1[:,1:]
#imagestest = np.multiply(imagestest, 1.0 / 255.0)
#images = images.astype(np.float64)
labls = tmp1[:,0]
lablstest = tmptest1[:,0]

labels=labls.reshape(len(labls),1)
labelstest=lablstest.reshape(len(lablstest),1)

def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

labels = dense_to_one_hot(labls, 10)
labels = labels.astype(np.uint8)
labelstest = dense_to_one_hot(lablstest, 10)
labelstest = labelstest.astype(np.uint8)

GETTING 1000 samples
GETTING 1000 samples


## - Function to pick batch size 10 data from Training and Test dataset for Tensor Flow Session

In [27]:
epochs_completed = 0
index_in_epoch = 0
num_examples = images.shape[0]

# serve data by batches
def next_batch(batch_size):
    
    global images
    global labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        images = images[perm]
        labels = labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return images[start:end], labels[start:end]

In [28]:
epochs_completed = 0
index_in_epoch = 0
num_examples = images.shape[0]

# serve data by batches
def next_batch_Te(batch_size):
    
    global imagestest
    global labelstest
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        images = images[perm]
        labels = labels[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return imagestest[start:end], labelstest[start:end]

## -------------COMMON CODE ENDS ----------------##
# - TF Code to run Neural Network for "Problem 2B" with below paramters
## - 30 Hidden Sigmoid Nodes, 2 Hidden Layers, No L2 Regularisation and 10 Softmax output nodes 
### - Learning Rate=0.1, Number of Epochs= 30, Batch Size = 10
### - Use the scoping of all TF Parameters (Arrays and Scalar) for Display in Tensor Board

In [11]:
learning_rate = 0.1
training_epochs = 30
batch_size = 10
display_step = 2
beta = 5
logdir = '/Users/vikrant/GoogleDrive/CSE555-Intro2Pattern-WenDong/HW-45/'
with tf.name_scope('input'):#as scope:
    # None -> batch size can be any size, 784 -> flattened mnist image
   
    x = tf.placeholder(tf.float32, shape=[None, 784], name="Input") 
with tf.name_scope('output'):#as scope:    
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Output")

# Create a model
with tf.name_scope("Hidden_Layer"):# as scope:
    W1 = tf.Variable(tf.random_normal([784, 30], stddev=0.03), name='W1_weights')
    b1 = tf.Variable(tf.random_normal([30]), name='b1_biases')
    
    W1_h = tf.summary.histogram("W1_weights", W1)
    b1_h = tf.summary.histogram("b1_biases", b1)
    
    hidden_out = tf.add(tf.matmul(x,W1),b1)
    hidden_out = tf.nn.sigmoid(hidden_out, name = "H_Activation")
    hidden_h = tf.summary.histogram("Hidden_Activation", hidden_out)

with tf.name_scope('Hidden_Layer2'):
    Wh2 = tf.Variable(tf.random_normal([30, 30], stddev=0.03), name='Wh2_weights')#Between Hidden Layer1&2
    bh2 = tf.Variable(tf.random_normal([30]), name='bh2_biases')
                      
    Wh2_h = tf.summary.histogram("Wh2_weights", Wh2)
    bh2_h = tf.summary.histogram("bh2_biases", bh2)
                      
    hidden2_out = tf.add(tf.matmul(hidden_out,Wh2),bh2)
    hidden2_out = tf.nn.sigmoid(hidden2_out)
    hidden_h = tf.summary.histogram("Hidden2_Activation", hidden2_out)                  
                      
with tf.name_scope("Output_Layer"):# as scope:
    W2 = tf.Variable(tf.random_normal([30, 10], stddev=0.03), name='W2_weights')# and the weights connecting the hidden layer to the output layer
    b2 = tf.Variable(tf.random_normal([10]), name='b2_biases')
    W2_h = tf.summary.histogram("W2_weights", W2)
    b2_h = tf.summary.histogram("b2_biases", b2)
    
    logits = tf.matmul(hidden2_out,W2) + b2
    y = tf.nn.softmax(logits)
    softmax_h = tf.summary.histogram("Softmax_Activation", y)
    print("softmax:", y)

with tf.name_scope("optimiser"):#as scope:
    with tf.name_scope("Error"):
        #error_loss= tf.reduce_mean(tf.squared_difference(y, y_))
        error_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))#,reduction_indices=[1] )
        #regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
        error_loss = tf.reduce_mean(error_loss)# + beta * regularizers) 
        erro_h= tf.summary.histogram("Error", error_loss)
        #tf.summary.scalar("regularizer", regularizers)
        tf.summary.scalar("cost", error_loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(error_loss)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(error_loss)
# define an accuracy assessment operation
with tf.name_scope("Accuracy"):#as scope:
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_h =tf.summary.histogram("Accuracy", accuracy)
    tf.summary.scalar("accuracy_train", accuracy)
    
#with tf.name_scope("Learning_rate"):
   #p_lr = tf.placeholder(tf.float32, (), name='learning_rate')
    
    

test_summary = tf.summary.scalar("test_accuracy", accuracy)

summary_op = tf.summary.merge_all()

softmax: Tensor("Output_Layer/Softmax:0", shape=(?, 10), dtype=float32)


In [12]:
with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # create log writer object
    train_writer = tf.summary.FileWriter(logdir+'/tflog_pcip_h2_ce_woL2'+ '/train', graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(logdir+'/tflog_pcip_h2_ce_woL2'+ '/test', graph=tf.get_default_graph())    
    # perform training cycles
    # create log writer object
    #writer = tf.summary.FileWriter('/Users/vikrant/GoogleDrive/CSE555-Intro2Pattern-WenDong/HW-45/tflogsAdam/5', graph=tf.get_default_graph())
        
    # perform training cycles
    for iteration in range(training_epochs):
        
        # number of batches in one epoch
        #batch_count = int(len(y_train)/batch_size)
        batch_count = int(len(labels)/batch_size)
        for i in range(batch_count):
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_x, batch_y = images[i:i+10], labels[i:i+10]
            batch_x, batch_y = next_batch(batch_size)
            #batch_x, batch_y = x_train[i:i+10], y_train[i:i+10]
            #batch_x=formatdata(batch_x)
            #show_digits(batch_x, batch_y, "Digits{}")
            #batch_xs, batch_ys = x_test[i:i+10], y_test[i:i+10]
            batch_xs, batch_ys = next_batch_Te(batch_size)
            #batch_xs=formatdata(batch_xs)
            # perform the operations we defined earlier on batch
            _, summary = sess.run([optimizer, summary_op], feed_dict={x: batch_x, y_: batch_y})
           # _, l, a = sess.run([optimizer, error_loss, accuracy], feed_dict={x: batch_x, y_: batch_y})
           # _, test_summary = sess.run([optimizer, summary_op], feed_dict={x: batch_xs, y_: batch_ys})
            summary_test, acc = sess.run([summary_op, accuracy], feed_dict={x: batch_xs, y_: batch_ys})
           
            train_writer.add_summary(summary, iteration * batch_count + i)
            test_writer.add_summary(summary_test, iteration * batch_count + i)
            # write log
            #print("iteration:", iteration, "cost:",l,"train_accuracy:", a)
            #writer.add_summary(summary, iteration * batch_count + i)
            #writer.add_summary(test_summary,iteration * batch_count + i)
            #print("iteration * batch_count + i:", iteration * batch_count + i)
        if iteration % 5 == 0: 
            print ("Epoch: ", iteration) 
    #print ("Accuracy: ", accuracy.eval(feed_dict={x: x_test, y_: y_test}))
    print ("Accuracy: ", accuracy.eval(feed_dict={x: imagestest, y_: labelstest}))
    print(sess.run(accuracy, feed_dict={x: imagestest, y_: labelstest}))
    #print(sess.run(accuracy, feed_dict={x: x_test, y_: y_test}))
    print ("done")

Epoch:  0
Epoch:  5
Epoch:  10
Epoch:  15
Epoch:  20
Epoch:  25
Accuracy:  0.274
0.274
done


### -(For Graphs Please see the Report)-###
# -----------------------------------------------------------------------------#
# Q 2 B. NEURAL NETWORK with 30 Hidden Sigmoid Nodes, L2 Regularisation and 10 Softmax output nodes 
# -----------------------------------------------------------------------------#
### - Import Libraries including TensorFlow
## - TF Code to run Neural Network for "problem 2B" with below paramters
### - 30 Hidden Sigmoid Nodes, 2 Hidden Nodes , L2 Regularisation and 10 Softmax output nodes
### - Learning Rate=0.1, Number of Epochs= 30, Batch Size = 10, L2 Regularisation = 5
### - Use the scoping of all TF Parameters (Arrays and Scalar) for Display in Tensor Board

# ERROR Fn. CROSS ENTROPY,H2, L2 Regularisation, I/P: MyPC

In [29]:
learning_rate = 0.1
training_epochs = 30
batch_size = 10
display_step = 2
beta = 5
logdir = '/Users/vikrant/GoogleDrive/CSE555-Intro2Pattern-WenDong/HW-45/'
with tf.name_scope('input'):#as scope:
    # None -> batch size can be any size, 784 -> flattened mnist image
   
    x = tf.placeholder(tf.float32, shape=[None, 784], name="Input") 
with tf.name_scope('output'):#as scope:    
    # target 10 output classes
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name="Output")

# Create a model
with tf.name_scope("Hidden_Layer"):# as scope:
    W1 = tf.Variable(tf.random_normal([784, 30], stddev=0.03), name='W1_weights')
    b1 = tf.Variable(tf.random_normal([30]), name='b1_biases')
    
    W1_h = tf.summary.histogram("W1_weights", W1)
    b1_h = tf.summary.histogram("b1_biases", b1)
    
    hidden_out = tf.add(tf.matmul(x,W1),b1)
    hidden_out = tf.nn.sigmoid(hidden_out, name = "H_Activation")
    hidden_h = tf.summary.histogram("Hidden_Activation", hidden_out)

with tf.name_scope('Hidden_Layer2'):
    Wh2 = tf.Variable(tf.random_normal([30, 30], stddev=0.03), name='Wh2_weights')#Between Hidden Layer1&2
    bh2 = tf.Variable(tf.random_normal([30]), name='bh2_biases')
                      
    Wh2_h = tf.summary.histogram("Wh2_weights", Wh2)
    bh2_h = tf.summary.histogram("bh2_biases", bh2)
                      
    hidden2_out = tf.add(tf.matmul(hidden_out,Wh2),bh2)
    hidden2_out = tf.nn.sigmoid(hidden2_out)
    hidden_h = tf.summary.histogram("Hidden2_Activation", hidden2_out)                  
                      
with tf.name_scope("Output_Layer"):# as scope:
    W2 = tf.Variable(tf.random_normal([30, 10], stddev=0.03), name='W2_weights')# and the weights connecting the hidden layer to the output layer
    b2 = tf.Variable(tf.random_normal([10]), name='b2_biases')
    W2_h = tf.summary.histogram("W2_weights", W2)
    b2_h = tf.summary.histogram("b2_biases", b2)
    
    logits = tf.matmul(hidden2_out,W2) + b2
    y = tf.nn.softmax(logits)
    softmax_h = tf.summary.histogram("Softmax_Activation", y)
    print("softmax:", y)

with tf.name_scope("optimiser"):#as scope:
    with tf.name_scope("Error"):
        #error_loss= tf.reduce_mean(tf.squared_difference(y, y_))
        error_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_))#,reduction_indices=[1] )
        regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2)
        error_loss = tf.reduce_mean((error_loss) + beta * regularizers) 
        erro_h= tf.summary.histogram("Error", error_loss)
        tf.summary.scalar("regularizer", regularizers)
        tf.summary.scalar("cost", error_loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(error_loss)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(error_loss)
# define an accuracy assessment operation
with tf.name_scope("Accuracy"):#as scope:
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    accuracy_h =tf.summary.histogram("Accuracy", accuracy)
    tf.summary.scalar("accuracy_train", accuracy)
    
#with tf.name_scope("Learning_rate"):
   #p_lr = tf.placeholder(tf.float32, (), name='learning_rate')
    
    

test_summary = tf.summary.scalar("test_accuracy", accuracy)

summary_op = tf.summary.merge_all()

softmax: Tensor("Output_Layer/Softmax:0", shape=(?, 10), dtype=float32)


In [30]:
with tf.Session() as sess:
    # variables need to be initialized before we can use them
    sess.run(tf.global_variables_initializer())

    # create log writer object
    train_writer = tf.summary.FileWriter(logdir+'/tflog_pcip_h2_ce_wL2'+ '/train', graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter(logdir+'/tflog_pcip_h2_ce_wL2'+ '/test', graph=tf.get_default_graph())    
    # perform training cycles
    # create log writer object
    #writer = tf.summary.FileWriter('/Users/vikrant/GoogleDrive/CSE555-Intro2Pattern-WenDong/HW-45/tflogsAdam/5', graph=tf.get_default_graph())
        
    # perform training cycles
    for iteration in range(training_epochs):
        
        # number of batches in one epoch
        #batch_count = int(len(y_train)/batch_size)
        batch_count = int(len(labels)/batch_size)
        for i in range(batch_count):
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_x, batch_y = images[i:i+10], labels[i:i+10]
            batch_x, batch_y = next_batch(batch_size)
            #batch_x, batch_y = x_train[i:i+10], y_train[i:i+10]
            #batch_x=formatdata(batch_x)
            #show_digits(batch_x, batch_y, "Digits{}")
            #batch_xs, batch_ys = x_test[i:i+10], y_test[i:i+10]
            batch_xs, batch_ys = next_batch_Te(batch_size)
            #batch_xs=formatdata(batch_xs)
            # perform the operations we defined earlier on batch
            _, summary = sess.run([optimizer, summary_op], feed_dict={x: batch_x, y_: batch_y})
           # _, l, a = sess.run([optimizer, error_loss, accuracy], feed_dict={x: batch_x, y_: batch_y})
           # _, test_summary = sess.run([optimizer, summary_op], feed_dict={x: batch_xs, y_: batch_ys})
            summary_test, acc = sess.run([summary_op, accuracy], feed_dict={x: batch_xs, y_: batch_ys})
           
            train_writer.add_summary(summary, iteration * batch_count + i)
            test_writer.add_summary(summary_test, iteration * batch_count + i)
            # write log
            #print("iteration:", iteration, "cost:",l,"train_accuracy:", a)
            #writer.add_summary(summary, iteration * batch_count + i)
            #writer.add_summary(test_summary,iteration * batch_count + i)
            #print("iteration * batch_count + i:", iteration * batch_count + i)
        if iteration % 5 == 0: 
            print ("Epoch: ", iteration) 
    #print ("Accuracy: ", accuracy.eval(feed_dict={x: x_test, y_: y_test}))
    print ("Accuracy: ", accuracy.eval(feed_dict={x: imagestest, y_: labelstest}))
    print(sess.run(accuracy, feed_dict={x: imagestest, y_: labelstest}))
    #print(sess.run(accuracy, feed_dict={x: x_test, y_: y_test}))
    print ("done")

Epoch:  0
Epoch:  5
Epoch:  10
Epoch:  15
Epoch:  20
Epoch:  25
Accuracy:  0.1
0.1
done


### -(For Graphs Please see the Report)-###

# -----------------------------------------------------------------------------#